# Introducción a SQL
***
<div class="panel panel-danger">
    <div class='panel-heading'>
    <h4>Antes de empezar</h4>
    </div>
    <div class='panel-body'>
    <p>Para poder ejecutar queries a la base de datos, necesitamos tener instalado antes el paquete [ipython-sql](https://github.com/catherinedevlin/ipython-sql).
    
    <p>Este paquete nos permite escribir las queries sql en las celdas del notebook y visualizar el resultado como tablas html renderizadas
    <p>Ejecuta `!pip install ipython-sql` si es la primera vez que usas este notebook
    </div>
</div>

In [ ]:
!pip install ipython-sql

Una vez instalado, podemos invocar el entorno `SQL` usando `%load_ext sql`.
Esto nos permitira usar el magic de ipython `%%sql` al principo de **cada** celda y ejecutar comandos sql directamente

In [ ]:
# Esto carga el paquete instalado y nos conecta con la base de datos contenida en el fichero "bdd_notas.sqlite"
%load_ext sql
%sql sqlite:///bdd_notas.sqlite

Vamos a comprobar que todo es correcto pidiendole que nos enseñe los nombres de las tablas en la BDD

In [ ]:
%%sql

SELECT name FROM sqlite_master WHERE type='table';


<div class="panel panel-success">
    <div class='panel-heading'>
    <h4>Empecemos</h4>
    </div>
</div>


Los objetivos de este notebook son:

1. Entender la sintaxis SQL básica
2. Entender la diferencia entre "inner", “left”, “right” y “full” joins
3. Entender groupings y como se conectan a las funciones de agregación
4. Como ejecutar queries SQL y guardar los resultados en pandas.DataFrame()